In [5]:
#imports
import experiments as exp
experiment_runs = 10000

This part is MLS. It runs the FM local search 10.000 times.

In [6]:
from IPython.display import HTML
import pandas as pd
import time

#Run the multi start local search algorithm 10.000 times. The return value is a list of dictionaries.
#Each dictionary contains the results of one full FM run. 
#Each dictionary contains the following keys:
"""
"fm_runs": number of runs until convergence,
"run_times": a list of the run times of each run,
"total_elapsed": total time elapsed,
"average_elapsed": average time elapsed,
"cut_size": best cut size found,
"partition_1": a list of node ids in partition 1,
"partition_2": a list of node ids in partition 2,
"initial_cut_size": initial cut size
"""
start_time = time.time()
results = exp.run_mls(runs=experiment_runs)
end_time = time.time()

elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds to run {experiment_runs} experiments")

Elapsed time: 458.80 seconds to run 10000 experiments


In [8]:
summary = exp.summarize_results(results)

# Convert summary to DataFrame for better display
df = pd.DataFrame(list(summary.items()), columns=['Metric', 'Value'])
HTML(df.to_html(index=False))

Metric,Value
Total Runs,10000.000000
Average Runs,501.000000
Average Elapsed (full FM),0.044757
Stdev Elapsed (full FM),0.002826
Total Elapsed,447.565141
Average Cut Size,68.745800
Best Cut Size,29.000000
Worst Cut Size,118.000000
Average Initial Cut Size,642.225300
Best Initial Cut Size,560.000000


This is the section to find an optimum mutation size for the ILS. 
WARNING, it is not parallelized yet, it can take up to 15 hours!

In [10]:
import ils
import time

#Use the bisection search from the previous step to find the best mutation size for the ILS algorithm.
mutation_size_sequence = []
current = 10
lower_bound = current
max = 101
max_iterations = 10000
results = []
best_cut = 501
best_mutation_size = -1
tolerance = 2
n_worse_solutions = 0
step = 10

while current <= max:
    mutation_size_sequence.append(current)
    m_size = current      
    
    #Run the algorithm with the current mutation size 10 times.
    print("Running ILS with mutation size: ", m_size,". Trial: ", len(mutation_size_sequence))  
    start = time.time()
    best, avg_cut_size, results = ils.run_ils(mutation_size=m_size, max_iterations=max_iterations, runs=10)    
    time_elapsed = time.time() - start
    results.append((m_size, results))
    is_better_cut = avg_cut_size < best_cut
    
    if is_better_cut:
        best_cut = avg_cut_size
        best_mutation_size = m_size        
        n_worse_solutions = 0
    else:
        n_worse_solutions += 1 #If tolerance is not exceeded, we will keep exploring.
        
    print("Finished. Cut size: ", avg_cut_size, ". Is better: ", is_better_cut)
    print("Elapsed Time: ", time_elapsed)    
    print("--------------------------------------------")
    
    if n_worse_solutions > tolerance:
        print("Stopping. There is no improvement by incresing mutation size since", tolerance, "trials.")
        break
    
    current += step
    
print("Best mutation size: ", best_mutation_size)
print("Best cut size: ", best_cut)
#Fine tune the mutation size, by running the ILS with mutation size backwards one by one. But only if the best mutation size is greater than 5.
tune_val = best_mutation_size
n_worse_solutions = 0 #reset the counter.
for i in range(2, step+2,2):
    m_size = tune_val - i
    if m_size < 1:
        break    
    
    print("Fine tuning ILS with mutation size: ", m_size,". Trial: ", len(mutation_size_sequence))    
    start = time.time()
    best, avg_cut_size, results = ils.run_ils(mutation_size=m_size, max_iterations=max_iterations, runs=10)    
    time_elapsed = time.time() - start
    results.append((m_size, results))
    is_better_cut = avg_cut_size < best_cut    
    
    if is_better_cut:
        best_cut = avg_cut_size
        best_mutation_size = m_size        
        n_worse_solutions = 0
    else:
        n_worse_solutions += 1 #If tolerance is not exceeded, we will keep exploring.
        
    print("Finished. Cut size: ", avg_cut_size, ". Is better: ", is_better_cut)
    print("Elapsed Time: ", time_elapsed)    
    print("--------------------------------------------")
    
    if n_worse_solutions > tolerance:
        print("Stopping. There is no improvement by increasing mutation size since", tolerance, "trials.")
        break
    

Running ILS with mutation size:  10 . Trial:  1
ILS -0- Mutation Size: 10. Best Cut: 29.
ILS -1- Mutation Size: 10. Best Cut: 23.
ILS -2- Mutation Size: 10. Best Cut: 25.
ILS -3- Mutation Size: 10. Best Cut: 29.
ILS -4- Mutation Size: 10. Best Cut: 30.
ILS -5- Mutation Size: 10. Best Cut: 24.
ILS -6- Mutation Size: 10. Best Cut: 21.
ILS -7- Mutation Size: 10. Best Cut: 21.
ILS -8- Mutation Size: 10. Best Cut: 48.
ILS -9- Mutation Size: 10. Best Cut: 20.
Finished. Cut size:  27 . Is better:  True
Elapsed Time:  8333.888215065002
--------------------------------------------
Running ILS with mutation size:  20 . Trial:  2
ILS -0- Mutation Size: 20. Best Cut: 11.
ILS -1- Mutation Size: 20. Best Cut: 21.
ILS -2- Mutation Size: 20. Best Cut: 35.
ILS -3- Mutation Size: 20. Best Cut: 21.
ILS -4- Mutation Size: 20. Best Cut: 19.
ILS -5- Mutation Size: 20. Best Cut: 31.
ILS -6- Mutation Size: 20. Best Cut: 18.
ILS -7- Mutation Size: 20. Best Cut: 26.
ILS -8- Mutation Size: 20. Best Cut: 20.
ILS 

In [ ]:
import mls
best_cut, summary, detailed_results = mls.run_mls(max_iterations=10000,runs=10,graph_file="Graph500.txt")

MLS - 0. Best Cut: 29. Elapsed: 446.283.
MLS - 1. Best Cut: 28. Elapsed: 447.087.
MLS - 2. Best Cut: 23. Elapsed: 447.709.
MLS - 3. Best Cut: 27. Elapsed: 448.844.
MLS - 4. Best Cut: 26. Elapsed: 450.055.
MLS - 5. Best Cut: 32. Elapsed: 448.925.
MLS - 6. Best Cut: 28. Elapsed: 450.549.
MLS - 7. Best Cut: 30. Elapsed: 453.365.
